In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# Load the dataset
hr_data = pd.read_csv('Human Resources.csv')

In [3]:
# Display the first few rows of the dataset and its structure
hr_data.head()

,id,first_name,last_name,birthdate,gender,race,department,jobtitle,location,hire_date,termdate,location_city,location_state
0,00-0037846,Kimmy,Walczynski,06-04-91,Male,Hispanic or Latino,Engineering,Programmer Analyst I,Headquarters,1/20/2002,NaN,Cleveland,Ohio
1,00-0041533,Ignatius,Springett,6/29/1984,Male,White,Business Development,Business Analyst,Headquarters,04-08-19,NaN,Cleveland,Ohio
2,00-0045747,Corbie,Bittlestone,7/29/1989,Male,Black or African American,Sales,Solutions Engineer Manager,Headquarters,10-12-10,NaN,Cleveland,Ohio
3,00-0055274,Baxy,Matton,9/14/1982,Female,White,Services,Service Tech,Headquarters,04-10-05,NaN,Cleveland,Ohio
4,00-0076100,Terrell,Suff,04-11-94,Female,Two or More Races,Product Management,Business Analyst,Remote,9/29/2010,2029-10-29 06:09:38 UTC,Flint,Michigan


In [4]:
hr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22214 entries, 0 to 22213
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              22214 non-null  object
 1   first_name      22214 non-null  object
 2   last_name       22214 non-null  object
 3   birthdate       22214 non-null  object
 4   gender          22214 non-null  object
 5   race            22214 non-null  object
 6   department      22214 non-null  object
 7   jobtitle        22214 non-null  object
 8   location        22214 non-null  object
 9   hire_date       22214 non-null  object
 10  termdate        3929 non-null   object
 11  location_city   22214 non-null  object
 12  location_state  22214 non-null  object
dtypes: object(13)
memory usage: 2.2+ MB


In [5]:
hr_data.describe()

,id,first_name,last_name,birthdate,gender,race,department,jobtitle,location,hire_date,termdate,location_city,location_state
count,22214,22214,22214,22214,22214,22214,22214,22214,22214,22214,3929,22214,22214
unique,22214,7758,17754,10854,3,7,13,185,2,7016,3929,77,7
top,00-0037846,Cassie,Ducker,05-09-72,Male,White,Engineering,Research Assistant II,Headquarters,07-06-09,2029-10-29 06:09:38 UTC,Cleveland,Ohio
freq,1,12,7,9,11288,6328,6686,754,16715,10,1,16871,18025


In [6]:
# Function to Standardize Date Format
def standardize_date(date_str, remove_tz=True):
    """
    Standardize a date string to a datetime object in 'YYYY-MM-DD' format.
    Timezone information is removed if 'remove_tz' is True.
    Returns NaT for unparseable dates.
    """
    try:
        date = pd.to_datetime(date_str, errors='coerce').normalize()
        return date.tz_localize(None) if remove_tz else date
    except:
        return pd.NaT

## Apply Date Standardization and Data Cleaning

In [7]:
for column in ['birthdate', 'hire_date', 'termdate']:
    hr_data[column] = hr_data[column].apply(standardize_date, remove_tz=column != 'hire_date')

In [8]:
hr_data.drop_duplicates(inplace=True)
hr_data.columns = hr_data.columns.str.replace(' ', '_')
hr_data[['first_name', 'last_name']] = hr_data[['first_name', 'last_name']].apply(lambda x: x.str.title())

In [9]:
missing_values = hr_data.isna().sum()
print("\nMissing Values:")
print(missing_values)


Missing Values:
id                    0
first_name            0
last_name             0
birthdate             0
gender                0
race                  0
department            0
jobtitle              0
location              0
hire_date             0
termdate          18285
location_city         0
location_state        0
dtype: int64


In [10]:
# Checking the uniqueness of the 'id' column
print("\nUnique ID Check:", hr_data['id'].is_unique)


Unique ID Check: True


In [11]:
# Calculate Age
current_date = pd.to_datetime('today')
hr_data['age'] = hr_data['birthdate'].apply(lambda bd: current_date.year - bd.year - ((current_date.month, current_date.day) < (bd.month, bd.day)) if pd.notna(bd) else None)

In [12]:
#Handle missing data (example - fill missing 'age' with median)
hr_data['age'].fillna(hr_data['age'].median(), inplace=True)

In [13]:
hr_data.describe()

,age
count,22214.000000
mean,30.593770
std,25.527303
min,-50.000000
25%,27.000000
50%,36.000000
75%,45.000000
max,58.000000


In [14]:
# Handle Negative or Unreasonably High Ages
hr_data = hr_data[hr_data['age'] >= 0] 

In [15]:
# Export Cleaned Data
hr_data.to_csv('Cleaned_HR_Data.csv', index=False)

In [16]:
# Display Cleaned Data Statistics, Types, and Preview
print("\nCleaned Data Statistics:")
print(hr_data.describe())
print("\nCleaned Data Types:")
print(hr_data.dtypes)
print("\nPreview of Cleaned Data:")
print(hr_data.head())


Cleaned Data Statistics:
                age
count  20304.000000
mean      37.836436
std       10.115415
min       21.000000
25%       29.000000
50%       37.000000
75%       46.000000
max       58.000000

Cleaned Data Types:
id                        object
first_name                object
last_name                 object
birthdate         datetime64[ns]
gender                    object
race                      object
department                object
jobtitle                  object
location                  object
hire_date         datetime64[ns]
termdate          datetime64[ns]
location_city             object
location_state            object
age                        int64
dtype: object

Preview of Cleaned Data:
           id first_name    last_name  birthdate  gender  \
0  00-0037846      Kimmy   Walczynski 1991-06-04    Male   
1  00-0041533   Ignatius    Springett 1984-06-29    Male   
2  00-0045747     Corbie  Bittlestone 1989-07-29    Male   
3  00-0055274       Baxy       